# Buidling data set to generate Z matrix, based on Goslin
---

This notebook generates data sets to build our Z matrices based on our model:       
    
        𝐘 = 𝐗 𝛃 𝐙' + 𝐄 , where    

- 𝐘 is the outcome data matrix (trigliceride levels)
- 𝐗 is a design matrix constructed from individual covariates (phenotype status, whether they were taking fish oil)
- 𝛃 is the coefficient matrices to estimate
- 𝐙 is a design matrix constructed from outcome covariates (e.g., information about the triglicerides)
- 𝐄 is random error assumed to have mean zero, independent across individuals, but possibly correlated across outcomes

Goslin is a Lipid Shorthand Nomenclature Grammars and uses ANTLRv4 compatible context-free EBNF grammars; we use the R package `rgoslin` to extract lipids information such as oxidation, number of double bonds... 

## Input

### Libraries

In [1]:
# To use RCall for the first time, one needs to 
# the location of the R home directory.
firstTimeRCall = false
if firstTimeRCall
    using Pkg
    io = IOBuffer()
    versioninfo(io)
    if occursin("Windows", String(take!(io)))
        ENV["R_HOME"] = "C:/PROGRA~1/R/R-44~1.0" # from R.home() in R
    else 
        ENV["R_HOME"] = "/usr/lib/R"

    end
    Pkg.build("RCall")
end         

In [1]:
using CSV, DataFrames, DataFramesMeta, Missings, CategoricalArrays
using StatsBase
# using StatsBase, Statistics, MultivariateStats, MatrixLM, Random, Distributions
using RCall
# using LinearAlgebra, RCall
# using FreqTables, Plots
# using StatsPlots

### Ext. Functions

In [2]:
include(joinpath(@__DIR__,"..", "..","src","utils.jl" ));

### Load data

Load look up table for the positive lipids.

In [3]:
# Load look up table `dfXref` contains the negative and the positve lipid ID (e.g., posLip1762) 
# and the original names of the potential identified lipids molecule (e.g. 4_TG(22:4_22:4_22:4)+NH4)
lipidsXref = realpath((@__DIR__)*"/../../data/data_processed/inl2b_Lipids_Xref.csv")
dfXref = DataFrame(CSV.File(lipidsXref));

## Extract first candidate of each lipids ID

Create a column containing the first cleaned candidate.

In [4]:
dfXref.Lipids = string.(map(x -> getLipName(x), dfXref.OriginalNames));

Let create extra columns that include information about oxydation ("Ox") and plasmogen information ("plasmanyl" or "plasmenyl"):

In [5]:
# Create oxydation and plasmogen column
dfXref.Oxidation = repeat(["no"], size(dfXref)[1]);
dfXref.Plasmalogen = repeat(["no"], size(dfXref)[1]);

Example:

In [6]:
# dfXref[sample(1:size(dfXref)[1], 3, replace= false),:] # 3 random lipids
dfXref[[495, 496, 696],:]

Row,lipID,OriginalNames,Lipids,Oxidation,Plasmalogen
,String15,String,String,String,String
1,posLip73,1_PC(14:0_16:0)+H,PC(14:0/16:0),no,no
2,posLip74,1_PC(14:0_20:4)+H,PC(14:0/20:4),no,no
3,posLip274,1_TG(16:0_20:4_22:6)+NH4,TG(16:0/20:4/22:6),no,no


We need to convert the nomenclature to be compatible with `GOSLIN` package nomenclature and fill in `Oxidation` and `Plasmalogen` columns:

In [7]:
# Clean Names 
for i in 1:size(dfXref)[1]
    dfXref.Lipids[i] = replace(dfXref.Lipids[i], "-H"=>"")
    dfXref.Lipids[i] = replace(dfXref.Lipids[i], "SPLASH_"=>"")
    dfXref.Lipids[i] = replace(dfXref.Lipids[i], "SPLASH/"=>"")
    if occursin("AcCar", dfXref.Lipids[i])
        dfXref.Lipids[i] = replace(dfXref.Lipids[i], "AcCar"=>"CAR" )
    else
        if occursin(r"(?i)plasmenyl-", dfXref.Lipids[i])
            dfXref.Lipids[i] = replace(dfXref.Lipids[i], r"(?i)plasmenyl-"=>"" )
            dfXref.Plasmalogen[i] = "plasmenyl"
        else
            if occursin(r"(?i)plasmanyl-", dfXref.Lipids[i])
            dfXref.Lipids[i] = replace(dfXref.Lipids[i], r"(?i)plasmanyl-"=>"" )
            dfXref.Plasmalogen[i] = "plasmanyl"
            else
                if occursin("Cer-NS", dfXref.Lipids[i])
                    dfXref.Lipids[i] = replace(dfXref.Lipids[i], "Cer-NS"=>"Cer" )
                else
                    if occursin("Ox", dfXref.Lipids[i])
                        dfXref.Lipids[i] = replace(dfXref.Lipids[i], "Ox"=>"")
                        dfXref.Oxidation[i] = "yes"
                    else
                        if occursin("AcCa(", dfXref.Lipids[i])
                                dfXref.Lipids[i] = replace(dfXref.Lipids[i], "AcCa("=>"CAR(" )
                        end
                    end
                end
            end
        end
    end
    
    
end

In [8]:
replace("TG(18:8_20:2_20:5)", "_" => "/")

"TG(18:8/20:2/20:5)"

### Save intermediate results

In [9]:
# realpath((@__DIR__)*"/../../data/dataprocessed/")

dfXref |> CSV.write("../../data/data_processed/dfXrefTest.csv")

"../../data/data_processed/dfXrefTest.csv"

## Build Z data sets

To build our raw Z dataset:
* we check the validity of the nomenclature grammar and filter invalid ones
* we filter the class of lipids of interest, for example we can keep only Triglycerides by filtering lipids containing the string "TG". 

### Build Z data set containing only Triglycerides

In [10]:
R"""
suppressMessages(library(rgoslin))
suppressMessages(library(tidyverse));
"""

RObject{StrSxp}
 [1] "lubridate" "forcats"   "stringr"   "dplyr"     "purrr"     "readr"    
 [7] "tidyr"     "tibble"    "ggplot2"   "tidyverse" "rgoslin"   "stats"    
[13] "graphics"  "grDevices" "utils"     "datasets"  "methods"   "base"     


In [11]:
@rput dfXref;

#### Lipids goup  preselection: triglycerides

In [12]:
R"""
# check validity
dfXref$Valid <- suppressWarnings(sapply(dfXref$Lipids, isValidLipidName))

# filter for valid and TG lipids
dfXref <- dfXref %>% filter(Valid) %>% filter(str_detect(Lipids, "TG"))

# get lipids information: total C and Double Bonds 
dfGoslin <- as_tibble(parseLipidNames(dfXref$Lipids))[, c("Original.Name", "Total.C", "Total.DB", "Lipid.Maps.Main.Class")];

# change columns name
names(dfGoslin) <- c("Lipids", "Total_C", "Total_DB", "Class")

# dfGoslin <- right_join(subset(dfXref, select = -c(Valid)), dfGoslin) # remove Valid col
dfGoslin <- cbind(subset(dfXref, select = -c(Valid)), subset(dfGoslin, select = c("Total_C", "Total_DB", "Class")))

"""
@rget dfGoslin;

We added two additional information: total number of carbon and total number of double bound.

In [15]:
# Parse Total_C and Total_DB
# dfGoslin.Total_C =parse.(Int64, dfGoslin.Total_C);
# dfGoslin.Total_DB =parse.(Int64, dfGoslin.Total_DB);

#### Save raw triglycerides Z matrix

In [16]:
dfGoslin |> CSV.write("../../data/data_processed/ZmatRawTG.csv")

"../../data/data_processed/ZmatRawTG.csv"

### Build Z matrix-PE, PC, PA

In [51]:
R"""
suppressMessages(library(rgoslin))
suppressMessages(library(tidyverse));
"""

RObject{StrSxp}
 [1] "forcats"   "stringr"   "dplyr"     "purrr"     "readr"     "tidyr"    
 [7] "tibble"    "ggplot2"   "tidyverse" "rgoslin"   "stats"     "graphics" 
[13] "grDevices" "utils"     "datasets"  "methods"   "base"     


In [52]:
@rput dfXref;

#### Lipids goup  preselection: phospholipids

In [57]:
R"""
# check validity
dfXref$Valid <- suppressWarnings(sapply(dfXref$Lipids, isValidLipidName))

# filter for valid and Phospholipids
dfXref <- dfXref %>% filter(Valid) %>% filter(str_detect(Lipids, "PE|PC|PA")) %>% 
            filter(!str_detect(Lipids, "TG")) %>% # remove Plasmalogen TG 
            filter(!str_detect(Lipids, "L")) # remove Lysophospholipids

# get lipids information: total C and Double Bonds 
dfGoslin <- as_tibble(parseLipidNames(dfXref$Lipids))[, c("Original Name", "Total C", "Total DB", "Lipid Maps Main Class")];

# change columns name
names(dfGoslin) <- c("Lipids", "Total_C", "Total_DB", "Class")

dfGoslin <- right_join(subset(dfXref, select = -c(Valid)), dfGoslin) # remove Valid col

"""
@rget dfGoslin;

Joining, by = "Lipids"


We added two additional information: total number of carbon and total number of double bound.

In [58]:
# Parse Total_C and Total_DB
dfGoslin.Total_C =parse.(Int64, dfGoslin.Total_C);
dfGoslin.Total_DB =parse.(Int64, dfGoslin.Total_DB);

#### Save raw phospholipids Z matrix

In [59]:
dfGoslin |> CSV.write("../../data/data_processed/ZmatRawPhos.csv")

"../../data/data_processed/ZmatRawPhos.csv"

### Build Z matrix- all lipids

In [11]:
R"""
suppressMessages(library(rgoslin))
suppressMessages(library(tidyverse));
"""

RObject{StrSxp}
 [1] "forcats"   "stringr"   "dplyr"     "purrr"     "readr"     "tidyr"    
 [7] "tibble"    "ggplot2"   "tidyverse" "rgoslin"   "stats"     "graphics" 
[13] "grDevices" "utils"     "datasets"  "methods"   "base"     


In [12]:
@rput dfXref;

#### Lipids goup  preselection: all lipids

In [13]:
R"""
# check validity
dfXref$Valid <- sapply(dfXref$Lipids, isValidLipidName)

print(dim(dfXref))

# filter for valid Lipids nomenclature
dfXref <- dfXref %>% filter(Valid)

print(dim(dfXref))
# get lipids information: total C and Double Bonds 
parseLipidNames(dfXref$Lipids[c(1:1)])#[, c("Original Name", "Total C", "Total DB", "Lipid Maps Main Class")];

# change columns name
# names(dfGoslin) <- c("Lipids", "Total_C", "Total_DB", "Class")
#print(dim(dfXref))


"""
# @rget dfGoslin;

┌ Warning: RCall.jl: Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'Cer-AS(d18:1/24:0)' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'Cer-NDS(d18:0/24:0)' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'Cer-NDS(d18:0/22:0)' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'LPC(18:1(d7))' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'LPE(18:1(d7))' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'NeuAcalpha2-3Galbeta1-4Glcbeta-Cer(d40:1)' caused an exception: Lipid not found
│ Warning in rcpp_is_valid_lipid_name(lipidName) :
│   Parsing of lipid name 'NeuAcalpha2-3Galbeta1-4Glcbeta-Cer(d34:1)' caused an exception: Lipid not foun

[1] 2186    6
[1] 2057    6


In [14]:
names(dfXref)

5-element Vector{String}:
 "lipID"
 "OriginalNames"
 "Lipids"
 "Oxidation"
 "Plasmalogen"

In [15]:
@rget dfXref

,lipID,OriginalNames
,String,String
1,negLip4,1_Cer-NS(d18:1/24:0)+HCO2 | 1_Cer-NS(d27:1/15:0)+HCO2
2,negLip5,1_Cer-NS(d18:2/24:0)+HCO2 | 1_Cer-NS(d18:1/24:1)+HCO2 | 1_Cer-NS(d20:2/22:0)+HCO2
3,negLip6,1_Cer-NS(d18:1/22:0)+HCO2 | 1_Cer-NS(d16:1/24:0)+HCO2 | 1_Cer-NS(d17:1/23:0)+HCO2
4,negLip7,1_Cer-NS(d18:1/23:0)+HCO2 | 1_Cer-NS(d17:1/24:0)+HCO2
5,negLip8,1_Cer-NS(d18:1/16:0)+HCO2
6,negLip9,1_Cer-NS(d18:2/24:1)+HCO2 | 1_Cer-NS(d18:1/24:2)+HCO2 | 1_Cer-NS(d20:2/22:1)+HCO2
7,negLip10,1_Cer-NS(d18:1/20:0)+HCO2 | 1_Cer-NS(d16:1/22:0)+HCO2
8,negLip11,1_Cer-NS(d18:2/22:0)+HCO2
9,negLip12,1_Cer-NS(d18:1/18:0)+HCO2


In [16]:
sum(dfXref.Valid)

2057

In [17]:
R"""
# check validity
dfXref$Valid <- suppressWarnings(sapply(dfXref$Lipids, isValidLipidName))
# filter for valid Lipids nomenclature
dfXref <- dfXref %>% filter(Valid)
# get lipids information: total; C and Double Bonds 
dfGoslin <- as_tibble(parseLipidNames(dfXref$Lipids))[, c("Original Name", "Total C", "Total DB", "Lipid Maps Main Class")];
# change columns name
names(dfGoslin) <- c("Lipids", "Total_C", "Total_DB", "Class")
dfGoslin <- right_join(subset(dfXref, select = -c(Valid)), dfGoslin) # remove Valid col

"""
@rget dfGoslin;

Joining, by = "Lipids"


┌ Warning: RCall.jl: Warning in rcpp_parse_lipid_name(lipidName) :
│   Parsing of lipid name 'FAHFA(18:3/18:1)' caused an exception: Inconsistancy in number of fatty acyl chains for lipid 'FAHFA'
│ Warning in rcpp_parse_lipid_name(lipidName) :
│   Parsing of lipid name 'FAHFA(18:3/18:0)' caused an exception: Inconsistancy in number of fatty acyl chains for lipid 'FAHFA'
│ Warning in rcpp_parse_lipid_name(lipidName) :
│   Parsing of lipid name 'FAHFA(18:3/18:2)' caused an exception: Inconsistancy in number of fatty acyl chains for lipid 'FAHFA'
│ Warning in (function (..., deparse.level = 1)  :
│   number of columns of result is not a multiple of vector length (arg 11)
└ @ RCall C:\Users\Fenril-Fractal\.julia\packages\RCall\iMDW2\src\io.jl:160


We added two additional information: total number of carbon and total number of double bound.

In [18]:
# Parse Total_C and Total_DB
dfGoslin.Total_C =parse.(Int64, dfGoslin.Total_C);
dfGoslin.Total_DB =parse.(Int64, dfGoslin.Total_DB);

#### Save raw Z matrix for all lipids

In [19]:
dfGoslin |> CSV.write("../../data/data_processed/ZmatRawAll.csv")

"../../data/data_processed/ZmatRawAll.csv"

In [35]:
countmap(dfGoslin.Class)

Dict{String, Int64} with 18 entries:
  "Glycerophosphoethanolamines [GP02]"                 => 74
  "Glycerophosphates [GP10]"                           => 53
  "Monoacylglycerophosphocholines [GP0105]"            => 69
  "Fatty acyl carnitines [FA0707]"                     => 25
  "Neutral glycosphingolipids [SP05]"                  => 9
  "Glycerophosphocholines [GP01]"                      => 410
  "Monoacylglycerols [GL0101]"                         => 10
  "Sterol esters [ST0102]"                             => 24
  "Ceramide phosphocholines (sphingomyelins) [SP0301]" => 25
  "Glycosyldiacylglycerols [GL0501]"                   => 64
  "Triacylglycerols [GL0301]"                          => 787
  "Diacylglycerols [GL0201]"                           => 45
  "Cholesterol and derivatives [ST0101]"               => 1
  "Ceramides [SP02]"                                   => 23
  "Monoacylglycerophosphoethanolamines [GP0205]"       => 20
  "Glycerophosphoglycerols [GP04]"              

In [79]:
versioninfo()

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i5-6500 CPU @ 3.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [80]:
R"""
sessionInfo()
"""

RObject{VecSxp}
R version 4.0.5 (2021-03-31)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 19043)

Matrix products: default

locale:
[1] LC_COLLATE=English_United States.1252 
[2] LC_CTYPE=English_United States.1252   
[3] LC_MONETARY=English_United States.1252
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.1252    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] forcats_0.5.1      stringr_1.4.0      dplyr_1.0.4        purrr_0.3.4       
 [5] readr_1.4.0        tidyr_1.1.2        tibble_3.1.1       ggplot2_3.3.3     
 [9] tidyverse_1.3.0    rgoslin_1.1.3.1000

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.6       cellranger_1.1.0 pillar_1.6.0     compiler_4.0.5  
 [5] dbplyr_2.1.0     tools_4.0.5      jsonlite_1.7.2   lubridate_1.7.10
 [9] lifecycle_1.0.0  gtable_0.3.0     pkgconfig_2.0.3  rlang_0.4.10    
[13] reprex_1.0.0     cli_